In [2]:
import folium
import numpy as np
import pandas as pd
from IPython.display import HTML

In [11]:
column_names = ['created_at', 'id', 'source', 'lat', 'long', 'text']
activity = pd.read_csv('output_city.tmp', header=None, names=column_names) \
             .append(pd.read_csv('output_geo.tmp', header=None, names=column_names)) \
             .drop_duplicates(subset='id') \
             .loc[lambda x: (x.lat >= 52.3216) & (x.lat <= 52.4251) & (x.long >= 4.7685) & (x.long <= 5.0173)]
print(activity.shape)
activity.head()

(195, 6)


,created_at,id,source,lat,long,text
0,Mon Jun 05 19:56:12 +0000 2017,871817952110284805,twitter,52.373100,4.892200,"Home for now, but I left a little piece of my ..."
1,Mon Jun 05 19:57:02 +0000 2017,871818162291044356,twitter,52.366670,4.900000,Good evening #Amsterdam! Sun will set in 30 mi...
3,Mon Jun 05 19:59:48 +0000 2017,871818861733191680,twitter,52.378300,4.929400,View from my bedroom. #home #amsterdam #ij #ja...
4,Mon Jun 05 20:00:20 +0000 2017,871818994927403008,twitter,52.373100,4.892200,"💌 @ Amsterdam, Netherlands https://t.co/8sdSQ..."
5,Mon Jun 05 20:01:34 +0000 2017,871819306568478721,twitter,52.378193,4.966624,De #schellingwouderbrug in #amsterdam ging zoj...


In [17]:
map = folium.Map(location=[52.3702, 4.8952], zoom_start=11)
for index, row in activity.iterrows():
    folium.Marker([row['lat'], row['long']]).add_to(map)
map